In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
house_data = pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')

In [ ]:
house_data.head()

In [ ]:
house_data.shape

In [ ]:
house_data.info()

In [ ]:
for column in house_data:
    print(house_data[column].value_counts())
    print("-"*80)

In [ ]:
house_data.isna().sum()

In [ ]:
house_data['society'].value_counts()

In [ ]:
house_data['society'].describe()

In [ ]:
house_data['society'] = house_data['society'].fillna('Blank')

In [ ]:
house_data['location'].value_counts()

In [ ]:
house_data['location'] = house_data['location'].fillna('Whitefield')

In [ ]:
house_data['size'].value_counts()

In [ ]:
house_data['size'] = house_data['size'].fillna('2 BHK')

In [ ]:
house_data['bath'].value_counts()

In [ ]:
house_data['bath'].describe()

In [ ]:
house_data['bath'] = house_data['bath'].fillna(house_data['bath'].median())

In [ ]:
house_data['balcony'].value_counts()

In [ ]:
house_data['balcony'].describe()

In [ ]:
house_data['balcony'] = house_data['balcony'].fillna(house_data['balcony'].median())

In [ ]:
house_data.isna().sum()

In [ ]:
house_data.info()

In [ ]:
house_data['size'].value_counts()

In [ ]:
house_data['size'].replace('4 Bedroom','4 BHK', inplace=True)

In [ ]:
house_data['size'].replace('1 RK','1 BHK', inplace=True)

In [ ]:
house_data['size'].value_counts()

In [ ]:
house_data['total_sqft'].unique()

In [ ]:
type(house_data['total_sqft'])

In [ ]:
def range_to_num(sqft):
    range = sqft.split('-')
    if len(range) == 2:
        return (float(range[0]) + float(range[1]))/2
    try:
        return float(sqft)
    except:
        return None

In [ ]:
house_data['total_sqft'] = house_data['total_sqft'].apply(range_to_num)

In [ ]:
house_data.head(n=50)

In [ ]:
house_data['bhk']=house_data['size'].str.split().str.get(0).astype(int)

In [ ]:
house_data.head(n=50)

In [ ]:
house_data.describe()

In [ ]:
house_data['location'].value_counts()

In [ ]:
house_data.value_counts()

In [ ]:
house_data['location'].value_counts()

In [ ]:
house_data['location'] = house_data['location'].apply(lambda x: x.strip())

In [ ]:
house_data['location'].value_counts()

In [ ]:
house_data_location_count = house_data['location'].value_counts()

In [ ]:
house_data_location_count_15 = house_data_location_count[house_data_location_count<=15]

In [ ]:
house_data_location_count_15

In [ ]:
house_data['location'] = house_data['location'].apply(lambda x: 'Other' if x in house_data_location_count_15 else x)

In [ ]:
house_data['location'].value_counts()

In [ ]:
house_data['society'].value_counts()

In [ ]:
house_data['society'].unique()

In [ ]:
house_data_society_count = house_data['society'].value_counts()

In [ ]:
house_data_society_count_15 = house_data_society_count[house_data_society_count<=15]

In [ ]:
house_data['society'] = house_data['society'].apply(lambda x: 'Other' if x in house_data_society_count_15 else x)

In [ ]:
house_data['society'].value_counts()

In [ ]:
house_data['availability'].value_counts()

In [ ]:
house_data['total_sqft'].describe()

In [ ]:
house_data['price_sqft'] = house_data['price'] * 100000 / house_data['total_sqft']

In [ ]:
house_data['price_sqft'].describe()

In [ ]:
house_data.describe()

In [ ]:
(house_data['total_sqft'] / house_data['bhk']).describe()

In [ ]:
house_data = house_data[((house_data['total_sqft'] / house_data['bhk'])>=300)]

In [ ]:
house_data.describe()

In [ ]:
house_data.shape

In [ ]:
def outliers_sqft(dframe):
    dataf_output = pd.DataFrame()
    for key, subdf in dframe.groupby('location'):
        M = np.mean(subdf.price_sqft)
        SD = np.std(subdf.price_sqft)
        
        gen_df = subdf[(subdf.price_sqft > (M-SD)) & (subdf.price_sqft <= (M+SD))]
        
        dataf_output = pd.concat([dataf_output,gen_df], ignore_index = True)
    return dataf_output
house_data = outliers_sqft(house_data)
house_data.describe()
        

In [ ]:
house_data['bhk'].describe()

In [ ]:
house_data['bhk'].value_counts()

In [ ]:
def outlier_bhk(datframe):
    exclude_index = np.array([])
    for loc, loc_df in datframe.groupby('location'):
        BHK_Stat = {}
        for bhk, bhk_dframe in loc_df.groupby('bhk'):
            BHK_Stat[bhk] = {
                'Mean': np.mean(bhk_dframe.price_sqft),
                'STDV': np.std(bhk_dframe.price_sqft),
                'Count': bhk_dframe.shape[0]
            }
        
        for bhk, bhk_dframe in loc_df.groupby('bhk'):
            Stat = BHK_Stat.get(bhk-1)
            if Stat and Stat['Count'] > 5:
                exclude_index = np.append(exclude_index, bhk_dframe[bhk_dframe.price_sqft < (Stat['Mean'])].index.values)
            
            return datframe.drop(exclude_index,axis='index')

In [ ]:
house_data = outlier_bhk(house_data)

In [ ]:
house_data.shape

In [ ]:
house_data

In [ ]:
house_data.drop(columns = ['size','price_sqft'], inplace=True)

In [ ]:
house_data.head()

In [ ]:
house_data['availability'].value_counts()

house_data['availability'].replace('18-Dec','Not Ready To Move', inplace=True)
house_data['availability'].replace('18-May','Not Ready To Move', inplace=True)
house_data['availability'].replace('18-Apr','Not Ready To Move', inplace=True)
house_data['availability'].replace('18-Aug','Not Ready To Move', inplace=True)
house_data['availability'].replace('16-Oct','Not Ready To Move', inplace=True)
house_data['availability'].replace('15-Dec','Not Ready To Move', inplace=True)
house_data['availability'].replace('15-Jun','Not Ready To Move', inplace=True)
house_data['availability'].replace('14-Jul','Not Ready To Move', inplace=True)
house_data['availability'].replace('15-Nov','Not Ready To Move', inplace=True)
house_data['availability'].replace('22-Jun','Not Ready To Move', inplace=True)
house_data['availability'].replace('20-Apr','Not Ready To Move', inplace=True)
house_data['availability'].replace('16-Jul','Not Ready To Move', inplace=True)
house_data['availability'].replace('16-Nov','Not Ready To Move', inplace=True)
house_data['availability'].replace('16-Mar','Not Ready To Move', inplace=True)


In [ ]:
house_data['availability'].value_counts()

In [ ]:
house_data.to_csv("House_data_clean.csv")

In [ ]:
X = house_data.drop(columns = ['price'])
Y = house_data['price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:
print(X_Train.shape)
print(X_Test.shape)

In [ ]:
Column_Trans = make_column_transformer((OneHotEncoder(sparse = False), ['location', 'area_type','availability','society']), remainder = 'passthrough')

In [ ]:
Scaler = StandardScaler()

In [ ]:
lrm = LinearRegression(normalize = True)

In [ ]:
pipe = make_pipeline(Column_Trans, Scaler, lrm)

In [ ]:
pipe.fit(X_Train, Y_Train)

In [ ]:
Y_Predict_lrm = pipe.predict(X_Test)

In [ ]:
r2_score(Y_Test, Y_Predict_lrm)